In [31]:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import gym # openAi gym
import datetime
from time import sleep
import random
import itertools
import numpy as np

import sys


In [33]:
env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [34]:
# Let's first do some random steps in the game so you see how the game looks like

rew_tot=0
obs= env.reset()
env.render()
for _ in range(6):
    action = env.action_space.sample() #take step using random action from possible actions (actio_space)
    obs, rew, done, info = env.step(action) 
    rew_tot = rew_tot + rew
    env.render()
#Print the reward of these random action
print("Reward: %r" % rew_tot)    

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Reward: -33


In [35]:
print(env.observation_space)
print()
env.s=49 # some random number, you might recognize it
env.render()
env.s = 450 # and some other
env.render()

Discrete(500)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


#Brute force approach

In [36]:
env.action_space.sample()

1

In [37]:
def Brute_force (env):
  env = gym.make("Taxi-v3").env
  #reset environment to random state
  env.reset() 
  print("Action Space {}".format(env.action_space))
  print("State Space {}".format(env.observation_space))
  env.render()
  #BruteForce Approach
  env.s=152
  epochs = 0
  penalties, reward = 0, 0
  global frames
  frames = [] 
  done = False
  while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1     
  print("Timesteps taken: {}".format(epochs)) 
  print("Penalties incurred: {}".format(penalties))

In [38]:
Brute_force(env)

Action Space Discrete(6)
State Space Discrete(500)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Timesteps taken: 1096
Penalties incurred: 357


#Q-learning algorithm

In [39]:
def qLearning(env,l=[0.1,0.95,0.1]):
  print("\n----------------------------------------------------------------------")
  %%time
  env = gym.make("Taxi-v3").env
  # Initialize the q table
  global Q
  Q = np.zeros([env.observation_space.n, env.action_space.n])
  episodes=[]
  print("Hyper Parameters ",l)
  # For plotting metrics
  all_epochs = []
  all_penalties = []

  for i in range(1, 10001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < l[2]:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(Q[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        old_value = Q[state, action]
        next_max = np.max(Q[next_state])
    
        new_value = (1 - l[0]) * old_value + l[0] * (reward + l[1] * next_max)
        Q[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
    if i % 1000 == 0:
        #clear_output(wait=True)
        l[0] -=.005
        l[2] -=.001
  
  print("Alpha",l[0])
  print("Gamma = ",l[1])
  print("Epsilon =",l[2])
  print("epochs",epochs)
  print("Training finished.\n")
  return Q

In [40]:
Q = qLearning(env)


----------------------------------------------------------------------
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.54 µs
Hyper Parameters  [0.1, 0.95, 0.1]
Alpha 0.04999999999999998
Gamma =  0.95
Epsilon = 0.09
epochs 13
Training finished.



In [41]:
Q

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.0252097 ,  1.90059144, -1.13649441,  2.28821047,  5.20997639,
        -5.31311473],
       [ 4.70062583,  6.01723947,  2.50203205,  5.71705632, 10.9512375 ,
        -1.9288565 ],
       ...,
       [-0.97779739,  7.53687   , -0.92499083, -0.93464908, -1.98844096,
        -2.81580834],
       [-2.77973455, -2.76036234, -2.56484467,  3.71021833, -4.26716432,
        -2.92713834],
       [-0.1995    , -0.2089525 ,  0.98662691, 17.55495355, -0.52302096,
        -1.        ]])

In [42]:
# Let's see how the algorithm solves the frozen-lakes game

rew_tot=0.
obs= env.reset()
done=False
while done != True: 
    action = np.argmax(Q[obs])
    obs, rew, done, info = env.step(action) #take step using selected action
    rew_tot += rew
    env.render()

print("Reward:", rew_tot) 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : :_|
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|

#Evaluate the agent

In [43]:
from numpy.lib.function_base import append
"""Evaluate agent's performance after Q-learning"""
def Evaluate_agent (Q):
  total_epochs, total_penalties = 0, 0
  totaltimesteps=[]
  episodes = 100

  for _ in range(episodes):
      state = env.reset()
      epochs, penalties, reward = 0, 0, 0
      
      done = False
      
      while not done:
          action = np.argmax(Q[state])
          state, reward, done, info = env.step(action)

          if reward == -10:
              penalties += 1

          epochs += 1

      total_penalties += penalties
      total_epochs += epochs
     
 
  print(f"\nResults after {episodes} episodes:")
  print(f"Average timesteps per episode: {total_epochs / episodes}")
  print(f"Average penalties per episode: {total_penalties / episodes}")
  totaltimesteps.append(total_epochs/episodes)

  return totaltimesteps
  

In [44]:
print(Evaluate_agent(Q))



Results after 100 episodes:
Average timesteps per episode: 13.03
Average penalties per episode: 0.0
[13.03]


#Grid search

In [67]:
# Define the parameters that used in your Grid Search
# with the list of values that you wish to try out

gamma = [.5,.6,.7]
alpha = [.5,.6,.7]
Epsilon  = [.5,.6,.7]

lists=[alpha,gamma,Epsilon]



In [68]:
lists

[[0.5, 0.6, 0.7], [0.5, 0.6, 0.7], [0.5, 0.6, 0.7]]

In [69]:

Averge_time=[]
print ("All lists: " + str(lists))
the_result = list(itertools.product(*lists)) 
print ("All permutations : " +  str(the_result))


All lists: [[0.5, 0.6, 0.7], [0.5, 0.6, 0.7], [0.5, 0.6, 0.7]]
All permutations : [(0.5, 0.5, 0.5), (0.5, 0.5, 0.6), (0.5, 0.5, 0.7), (0.5, 0.6, 0.5), (0.5, 0.6, 0.6), (0.5, 0.6, 0.7), (0.5, 0.7, 0.5), (0.5, 0.7, 0.6), (0.5, 0.7, 0.7), (0.6, 0.5, 0.5), (0.6, 0.5, 0.6), (0.6, 0.5, 0.7), (0.6, 0.6, 0.5), (0.6, 0.6, 0.6), (0.6, 0.6, 0.7), (0.6, 0.7, 0.5), (0.6, 0.7, 0.6), (0.6, 0.7, 0.7), (0.7, 0.5, 0.5), (0.7, 0.5, 0.6), (0.7, 0.5, 0.7), (0.7, 0.6, 0.5), (0.7, 0.6, 0.6), (0.7, 0.6, 0.7), (0.7, 0.7, 0.5), (0.7, 0.7, 0.6), (0.7, 0.7, 0.7)]


In [71]:
steps=[]
for i in range(27):
  Q_1=qLearning(env,list(the_result[i]))
  print("\n",Q_1,"\n")
  avg_timesteps=Evaluate_agent(Q_1)
  parameter_metric_dict={"alpha":the_result[i][0],"gamma":the_result[i][1],"Epsilon":the_result[i][2],"avg_timesteps":avg_timesteps}
  steps.append(parameter_metric_dict)
listoftimesteps=[]
for i in steps:
  avg_timesteps=i.get("avg_timesteps")
  listoftimesteps.append(avg_timesteps)

print("\nThe minimum time step is  ",steps[listoftimesteps.index(min(listoftimesteps))])  


----------------------------------------------------------------------
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
Hyper Parameters  [0.5, 0.5, 0.5]
Alpha 0.44999999999999996
Gamma =  0.5
Epsilon = 0.49
epochs 46
Training finished.


 [[  0.           0.           0.           0.           0.
    0.        ]
 [ -1.98925781  -1.97851562  -1.98925781  -1.97851562  -1.95703125
  -10.97851562]
 [ -1.828125    -1.65625     -1.828125    -1.65625     -1.3125
  -10.65625   ]
 ...
 [ -1.3125      -0.625       -1.3125      -1.65625    -10.3125
  -10.3125    ]
 [ -1.95703102  -1.91406279  -1.95703094  -1.9140625  -10.95703099
  -10.95703125]
 [  3.5          0.75         3.5          9.          -5.5
   -5.5       ]] 


Results after 100 episodes:
Average timesteps per episode: 13.18
Average penalties per episode: 0.0

----------------------------------------------------------------------
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Hyper Parameters  [0.5, 